## FDSN Waveform Downloader, Converter & Detector

Downloads I06AU and I52GB waveforms from IRIS FSDN services using Obpsy Mass Downloader and then converts them the .SAC file with it's stats for further processing. The .SAC files will be then further processed using "Bartlett" beamforming method and Adaptive F-Detector to identify signals most related to Anak Krakatau Volcanic Activity. These specific volcanic activity signals will be later ingested into the algorithm for the Deep Learning processing

This notebook consists of 3 parts:
1. Dowloading Data using Obspy Mass Downloader
2. Conversion of MSEED Files to .SAC
3. Conversion of .SAC files to CSV
4. Implementing FK Beamforming to each station group
5. Running Adaptive F-Detector to Isolate relevant Waveform

In [3]:
import sys
sys.path.append('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/')

import obspy
from obspy import UTCDateTime, read, read_inventory
from obspy.clients.fdsn.mass_downloader import GlobalDomain, \
    Restrictions, MassDownloader
from obspy.core.util.attribdict import AttribDict
import os
import subprocess
import glob
from tqdm.notebook import tqdm
from src.utils.converter import *
import concurrent.futures

### 1. Downloading Data Using Obspy Mass Downloader

Download the data ranging from 2018-06-24T00:00:00 until 2019-09-03T00:00:00

In [ ]:
# 1.1 I06AU Waveform Download

domain = GlobalDomain()

restrictions = Restrictions(
    # Get data for a whole year.
    starttime=obspy.UTCDateTime(2018, 6, 24),
    endtime=obspy.UTCDateTime(2019, 9, 3),
    # Chunk it to have one file per day.
    chunklength_in_sec=86400,
    network="IM", station="I06H*", location="", channel="BDF",
    # The typical use case for such a data set are noise correlations where
    # gaps are dealt with at a later stage.
    reject_channels_with_gaps=False,
    # Same is true with the minimum length. All data might be useful.
    minimum_length=0.0,
    # Guard against the same station having different names.
    minimum_interstation_distance_in_m=100.0)

mdl = MassDownloader(providers=["IRIS"])
mdl.download(domain, restrictions, mseed_storage="waveform_collection/I06AU/WAVEFORM_I06AU_MSEED",
             stationxml_storage="waveform_collection/I06AU/I06AU_STATIONS")

In [ ]:
# 1.2 I52GB Waveform Download

domain = GlobalDomain()

restrictions = Restrictions(
    starttime=obspy.UTCDateTime(2018, 6, 24),
    endtime=obspy.UTCDateTime(2019, 9, 3),
    chunklength_in_sec=86400,
    network="IM", station="I52H*", location="", channel="BDF",
    reject_channels_with_gaps=False,
    minimum_length=0.0,
    minimum_interstation_distance_in_m=100.0)


mdl = MassDownloader(providers=["IRIS"])
mdl.download(domain, restrictions, mseed_storage="waveform_collection/I52GB/WAVEFORM_I52GB_MSEED",
             stationxml_storage="waveform_collection/I52GB/I52GB_STATIONS")

### 2. Conversion of MSEED Files to .SAC
Converting MSEED Files to SAC Complete with Important Headers

In [4]:
# 2.1 I06AU Waveform Conversion

input_directory = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_MSEED'
output_directory = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_SAC'
stationxml_directory = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_STATIONS'

# Run the Function
mseed_to_sac(input_directory, output_directory, stationxml_directory)

In [ ]:
# 2.2 I52GB Waveform Conversion

input_directory = 'waveform_collection/I52GB/WAVEFORM_I52GB_MSEED'
output_directory = 'waveform_collection/I52GB/WAVEFORM_I52GB_SAC'
stationxml_directory = 'waveform_collection/I52GB/I52GB_STATIONS'

# Run the Function
mseed_to_sac(input_directory, output_directory, stationxml_directory)


### 3. Conversion of SAC files to CSV

In [4]:
# 3.1 I06AU SAC to csv conversion

folder_path = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_SAC'
output_folder = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_CSV'
freqmin = 0.7 
freqmax = 4.0 
df_grouped_file_paths = sac_path_grouping(folder_path)

# Iterate over the DataFrame rows and process each file path
with concurrent.futures.ProcessPoolExecutor() as executor:
    # Store future objects along with the corresponding file pattern for error handling
    future_to_file = {executor.submit(sac_to_csv, row['GroupedFilePath'], output_folder, freqmin, freqmax): row['GroupedFilePath'] for index, row in df_grouped_file_paths.iterrows()}

    # Process completed futures
    for future in tqdm(concurrent.futures.as_completed(future_to_file), total=len(future_to_file), desc="Processing SAC files"):
        file_path_pattern = future_to_file[future]
        try:
            # Get the result of the future
            csv_output_path = future.result()
            print(f'CSV file created at: {csv_output_path}')
        except Exception as e:
            print(f"Error processing file pattern {file_path_pattern}: {e}")

Processing SAC files:   0%|          | 0/436 [00:00<?, ?it/s]

CSV file created at: /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_CSV/IM.I06H_..BDF__20190205T000000Z__20190206T000000Z.csv


In [3]:
df_grouped_file_paths =sac_path_grouping('/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_SAC')
print(df_grouped_file_paths)

                                       GroupedFilePath
0    /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
1    /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
2    /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
3    /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
4    /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
..                                                 ...
431  /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
432  /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
433  /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
434  /run/media/viblab/Markov2/Haykal/AnakKrakatauE...
435  /run/media/viblab/Markov2/Haykal/AnakKrakatauE...

[436 rows x 1 columns]


In [ ]:
# 3.1 I52GB SAC to csv conversion

folder_path = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I06AU/WAVEFORM_I06AU_SAC'
output_folder = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I06AU/WAVEFORM_I06AU_CSV'
freqmin = 0.7 
freqmax = 4.0 
df_grouped_file_paths = sac_path_grouping(folder_path)

# Iterate over the DataFrame rows and process each file path
for index, row in df_grouped_file_paths.iterrows():
    file_path_pattern = row['GroupedFilePath']
    try:
        # Process each file path pattern with the sac_to_csv function
        csv_output_path = sac_to_csv(file_path_pattern, output_folder, freqmin, freqmax)
        print(f'CSV file created at: {csv_output_path}')
    except Exception as e:
        print(f"Error processing file pattern {file_path_pattern}: {e}")

### 4. Running Beamforming


In [ ]:
# 4.1 Running Beamforming on I06AU Waveform

base_folder_path = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I06AU/WAVEFORM_I06AU_SAC'
df_grouped_file_paths = sac_path_grouping(base_folder_path)

# Wrap the iterrows with tqdm to create a progress bar
for index, row in tqdm(df_grouped_file_paths.iterrows(), total=df_grouped_file_paths.shape[0], desc="Processing"):
    # Construct the full path to the grouped files
    full_grouped_path = f"{base_folder_path}/{row['GroupedFilePath']}"

    # Prepare the CLI command
    cli_command = f"infrapy run_fk --local-wvfrms \"{full_grouped_path}\" --"
    
    try:
        # Execute the CLI command
        subprocess.run(cli_command, shell=True, check=True)
        print(f"Command executed for: {full_grouped_path}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running the command for {full_grouped_path}: {e}")

In [ ]:
# 4.1 Running Beamforming on I52GB Waveform

base_folder_path = '/run/media/viblab/Markov11/Haykal/AnakKrakatauEWS/waveform_collection/I52GB/WAVEFORM_I52GB_SAC'
df_grouped_file_paths = sac_path_grouping(base_folder_path)

# Wrap the iterrows with tqdm to create a progress bar
for index, row in tqdm(df_grouped_file_paths.iterrows(), total=df_grouped_file_paths.shape[0], desc="Processing"):
    # Construct the full path to the grouped files
    full_grouped_path = f"{base_folder_path}/{row['GroupedFilePath']}"

    # Prepare the CLI command
    cli_command = f"infrapy run_fk --local-wvfrms \"{full_grouped_path}\" --"
    
    try:
        # Execute the CLI command
        subprocess.run(cli_command, shell=True, check=True)
        print(f"Command executed for: {full_grouped_path}")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while running the command for {full_grouped_path}: {e}")

In [5]:
csv_folder = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/raw/I06AU/I06AU_CSV'
detection_json = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/Uncofirmed_events.json'
output_folder = '/run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I06AU_NON_EVENT'

extract_det_from_csv(csv_folder, detection_json, output_folder)

Processing CSV files:   0%|          | 0/436 [00:00<?, ?it/s]

Processing CSV files:   0%|          | 1/436 [00:07<52:35,  7.25s/it]

Saved trimmed data to /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I06AU_NON_EVENT/IM_det_165_20181014_193905_to_20181014_193915.csv


Processing CSV files:   1%|          | 5/436 [00:38<55:25,  7.72s/it]

Saved trimmed data to /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I06AU_NON_EVENT/IM_det_136_20180809_001014_to_20180809_001019.csv
Saved trimmed data to /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I06AU_NON_EVENT/IM_det_137_20180809_015159_to_20180809_015204.csv
Saved trimmed data to /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I06AU_NON_EVENT/IM_det_138_20180809_072429_to_20180809_072434.csv
Saved trimmed data to /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I06AU_NON_EVENT/IM_det_139_20180809_073609_to_20180809_073614.csv


...

Processing CSV files:  92%|█████████▏| 402/436 [43:34<02:52,  5.07s/it]

Saved trimmed data to /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I06AU_NON_EVENT/IM_det_476_20190110_203650_to_20190110_203700.csv
Saved trimmed data to /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I06AU_NON_EVENT/IM_det_477_20190110_205010_to_20190110_205015.csv
Saved trimmed data to /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I06AU_NON_EVENT/IM_det_478_20190110_210115_to_20190110_210120.csv
Saved trimmed data to /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I06AU_NON_EVENT/IM_det_479_20190110_210415_to_20190110_210420.csv
Saved trimmed data to /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I06AU_NON_EVENT/IM_det_480_20190110_210650_to_20190110_210655.csv
Saved trimmed data to /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I06AU_NON_EVENT/IM_det_481_20190110_210825_to_20190110_210830.csv
Saved trimmed data to /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I0

Processing CSV files:  94%|█████████▍| 410/436 [44:05<02:03,  4.75s/it]

Saved trimmed data to /run/media/viblab/Markov2/Haykal/AnakKrakatauEWS/data/processed/I06AU_NON_EVENT/IM_det_535_20190215_175931_to_20190215_175936.csv


Processing CSV files: 100%|██████████| 436/436 [45:44<00:00,  6.29s/it]
